In [1]:
#output all identifiers including matched and unmached
import yaml
import requests
import json

with open(r'D:\BL\Renci\icees-api-config\Python-BL\Identifiers_all.yaml', 'w') as output_file:
    with open("identifiers.yml") as f:

            # open and read yam files
            data = yaml.load(f, Loader=yaml.FullLoader)

            tables_all = {}
            tables_matched = {}
            tables_unmatched = {}
            matched_output_file = open(r'D:\BL\Renci\icees-api-config\Python-BL\Identifiers_matched.yaml', 'w')
            unmatched_output_file = open(r'D:\BL\Renci\icees-api-config\Python-BL\Identifiers_unmatched.yaml', 'w')
            
            identifier_list = []
            for item, doc in data.items():
                for identifiers in doc.values():
                    identifier_list += identifiers

            identifier_set = set(identifier_list)
            input_obj = {
                "curies": list(identifier_set)
            }
            resp = requests.post("https://nodenormalization-sri.renci.org/1.1/get_normalized_nodes", headers={
                "Content-Type": "application/json",
                "Accept": "applicaton/json"
            }, json=input_obj)
            obj = resp.json()

            for item, doc in data.items():
                tables_all.setdefault(item, {})
                tables_matched.setdefault(item, {})
                tables_unmatched.setdefault(item, {})
                for variable in doc.keys():
                    identifiers = doc[variable]
                    
                    variable_identifiers_matched = []
                    variable_identifiers_unmatched = []
                    for identifier in identifiers:
                        if obj[identifier] is None:
                            variable_identifiers_unmatched.append({identifier : None})
                        else:
                            variable_identifiers_matched.append({identifier : str(obj[identifier]["id"]["identifier"])})
                    
                    tables_matched[item][variable] = variable_identifiers_matched
                    tables_unmatched[item][variable] = variable_identifiers_unmatched
                    tables_all[item][variable] = variable_identifiers_matched + variable_identifiers_unmatched
            yaml.dump(tables_matched, matched_output_file)
            yaml.dump(tables_unmatched, unmatched_output_file)
            yaml.dump(tables_all, output_file)
